<a href="https://colab.research.google.com/github/harnalashok/deeplearning-sequences/blob/main/network_connect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cdlib

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
#%reset -f

# 1.0 Call libraries
import pandas as pd
import numpy as np
import networkx as nx

# 1.01
import joblib
from collections import defaultdict

# 1.02
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# 1.08 Misc
import os,gc , time,  glob
from os.path import exists
import feather
import re
import ast


In [54]:
# Paths:
modulePath = "/gdrive/MyDrive/Colab_data_files/talkingData/03112022/"
dataPath = "/gdrive/MyDrive/Colab_data_files/skipgram/"
modelsPath = "/gdrive/MyDrive/Colab_data_files/skipgram/modelspath/"
pathToStoreProgress = "/gdrive/MyDrive/Colab_data_files/skipgram/pathToStoreProgress/"
mapdictPath = "/gdrive/MyDrive/Colab_data_files/skipgram/mapdictpath/"

In [55]:
# 1.09 Home made modules
os.chdir(modulePath)
import network_features
from network_features import *
import utils
from utils import *

import importlib; importlib.reload(network_features)
import importlib; importlib.reload(utils)

<module 'utils' from '/gdrive/MyDrive/Colab_data_files/talkingData/03112022/utils.py'>

In [56]:
# Cleaning up
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/modelspath/
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/pathToStoreProgress/
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/mapdictpath/

In [57]:
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/vectors.tsv
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/metadata.tsv

In [58]:

!mkdir /gdrive/MyDrive/Colab_data_files/skipgram/modelspath/
!mkdir /gdrive/MyDrive/Colab_data_files/skipgram/pathToStoreProgress/
!mkdir /gdrive/MyDrive/Colab_data_files/skipgram/mapdictpath/

In [ ]:
# 8.0 Read back pkl file and dictionary:
os.chdir(dataPath)
train = pd.read_pickle("seq.pkl")
train.head()

In [ ]:
# Read dictionary
os.chdir(dataPath)
filehandler = open("word_index.txt", 'r')
word2index = filehandler.read()
word2index

In [61]:
org_columns = train.columns
cat_cols = ['a', 'b']

In [62]:
# 6.1 Create new columns (combinations of columns) in train
train = utils.create_cat_pairwise_cols(train , cat_cols)

In [ ]:
train.head()

In [64]:
# 6.11 The train columns are:
current_train_columns = train.columns
# 6.12 New columns created. These columns must be latter dropped
new_cols = list(set(current_train_columns).difference(set(org_columns)))
# 6.13 Revised cat cols (ie excluding hour etc)
cat_cols.extend(new_cols)
len(cat_cols)   # 15
cat_cols

['a', 'b', 'a_p_b']

In [65]:
# 8.1 Get all possibe permutation of our columns
#     Also decide which all centrality measures to cal
deg = 1    # Calculate degree centrality
eig = 1    # Calculate eigenvector centrality
pr =  1    # Calculate pagerank centrality
clu = 0    # Clustering coeff centrality
nf = None # nx.katz_centrality_numpy
cMeasures = [deg,eig,pr,clu, nf]

In [66]:
single_permute_list = utils.permute_colList(cat_cols,cMeasures)
single_permute_list

[('a', 'b', [1, 1, 1, 0, None]),
 ('a', 'a_p_b', [1, 1, 1, 0, None]),
 ('b', 'a', [1, 1, 1, 0, None]),
 ('b', 'a_p_b', [1, 1, 1, 0, None]),
 ('a_p_b', 'a', [1, 1, 1, 0, None]),
 ('a_p_b', 'b', [1, 1, 1, 0, None])]

In [67]:
len(single_permute_list)   # 6

6

In [ ]:
spl = pd.DataFrame(single_permute_list, columns = ["colToProject", "intermediaryCol", "cMeasures?"])
spl.shape   # (210, 4)
spl

In [69]:
X = train

In [70]:
utils.ulevelsPerThreshold(X,'a_p_b', threshold_list = [0.00001, 0.0001, 0.0002, 0.000485,0.0005, 0.001, 0.01, 0.1, 0.15, 0.2])

,apply %threshold,remaining_levels
0,0.000000,184797
1,0.000010,184798
2,0.000100,184798
3,0.000200,184798
4,0.000485,13948
5,0.000500,13948
6,0.001000,3400
7,0.010000,33
8,0.100000,1
9,0.150000,1


In [71]:
mergethreshold = 0.00050  # 0.0020
X =  utils.levelsTransformer(X, 'a_p_b',mergethreshold, mapdictPath)

No saved dictionary exists as yet.


In [72]:
record_sets = list(spl.to_records(index = False))
record_sets

[('a', 'b', list([1, 1, 1, 0, None])),
 ('a', 'a_p_b', list([1, 1, 1, 0, None])),
 ('b', 'a', list([1, 1, 1, 0, None])),
 ('b', 'a_p_b', list([1, 1, 1, 0, None])),
 ('a_p_b', 'a', list([1, 1, 1, 0, None])),
 ('a_p_b', 'b', list([1, 1, 1, 0, None]))]

In [73]:
fulllist = [record_sets]

In [74]:
m = 1
for k,j in enumerate(fulllist):
  print("Record set no is {} of {}th fulllist ".format(k+1,m+1))
  print("Set of records is: ", j)
  # Each outer loop creates a new sets of models
  fulllist_repeat_no = m + 1
  transYesNo = False  # True implies transformation
  X1_train, X1_test = network_features.get_store_features(X,j, transYesNo, None, fulllist_repeat_no , modelsPath, pathToStoreProgress)
  print("\nList done. X1_train shape ", X1_train.shape)
  del X1_train
  gc.collect()

Record set no is 1 of 2th fulllist 
Set of records is:  [('a', 'b', list([1, 1, 1, 0, None])), ('a', 'a_p_b', list([1, 1, 1, 0, None])), ('b', 'a', list([1, 1, 1, 0, None])), ('b', 'a_p_b', list([1, 1, 1, 0, None])), ('a_p_b', 'a', list([1, 1, 1, 0, None])), ('a_p_b', 'b', list([1, 1, 1, 0, None]))]
('a', 'b', list([1, 1, 1, 0, None]))

Next:  1 a b
No of unique values in  a  are:  4044
Bipartite Network created. Time taken:  0.020483533541361492 minutes
Bipartite Network projected. Time taken:  0.3488311012585958 minutes
Model extracted. Time taken:  0.4066798408826192 minutes
Model stored to disk
train columns:  3
No test data
Done with a b
Time taken:  0.7940009872118632
Entry made in file progress.csv
('a', 'a_p_b', list([1, 1, 1, 0, None]))

Next:  2 a a_p_b
No of unique values in  a  are:  4044
Bipartite Network created. Time taken:  0.0066285371780395504 minutes
Bipartite Network projected. Time taken:  0.8190141002337138 minutes
Model extracted. Time taken:  1.1533069849014281 

In [75]:
# Update levels or merge rarely occurring levels
#  into one. Dictionaries for performing this mapping
#   are in folder: mapdictpath
X = utils.updateLevels(X,
                        mapdictPath,
                        cat_cols
                        )

In [76]:
def readStoredModels(pathToReadFrom, fileList):
  model_dfs = []
  os.chdir(pathToReadFrom)
  for file in fileList:
    df = pd.read_pickle(file)
    model_dfs.append(df)
  return model_dfs


In [77]:
os.chdir(modelsPath)
files = os.listdir()
file_list = [file for file in files if ("deg_a_ck" in file) or ("eig_a_ck" in file) or ("pr_a_ck" in file) ]


In [78]:
models = readStoredModels(modelsPath, file_list)

In [79]:
# Merge models
for i, model in enumerate(models):
  if i == 0:
    df = model
  else:
    df = pd.merge(df, model, left_index=True, right_index=True)  

In [80]:
df

,deg_a_ck_b,eig_a_ck_b,pr_a_ck_b,deg_a_ck_a_p_b,eig_a_ck_a_p_b,pr_a_ck_a_p_b
a,,,,,,
1.0,1.000000,0.033805,0.000691,1.0,0.015725,0.000247
2.0,1.000000,0.033805,0.000691,1.0,0.015725,0.000247
3.0,0.998269,0.033769,0.000690,1.0,0.015725,0.000247
4.0,0.995795,0.033714,0.000688,1.0,0.015725,0.000247
5.0,0.998516,0.033774,0.000690,1.0,0.015725,0.000247
...,...,...,...,...,...,...
4040.0,0.260945,0.012081,0.000198,1.0,0.015725,0.000247
4041.0,0.209251,0.009841,0.000167,1.0,0.015725,0.000247
4042.0,0.228296,0.010619,0.000179,1.0,0.015725,0.000247


In [81]:
# Save dataframe:
os.chdir(dataPath)
df.to_pickle("word2vec.pkl")


In [82]:
# Load word2vec
os.chdir(dataPath)
df_word= pd.read_pickle("word2vec.pkl")
df_word.head()

,deg_a_ck_b,eig_a_ck_b,pr_a_ck_b,deg_a_ck_a_p_b,eig_a_ck_a_p_b,pr_a_ck_a_p_b
a,,,,,,
1.0,1.000000,0.033805,0.000691,1.0,0.015725,0.000247
2.0,1.000000,0.033805,0.000691,1.0,0.015725,0.000247
3.0,0.998269,0.033769,0.000690,1.0,0.015725,0.000247
4.0,0.995795,0.033714,0.000688,1.0,0.015725,0.000247
5.0,0.998516,0.033774,0.000690,1.0,0.015725,0.000247


In [83]:
os.chdir(dataPath)
df_word.to_csv('vectors.tsv', sep="\t", index = False, header = False)

In [84]:
metadata = ast.literal_eval(re.search('({.+})', word2index).group(0))

In [ ]:
str_list = list(metadata.keys())
str_list

In [86]:
import sys
orig_stdout = sys.stdout
f = open('metadata.tsv', 'w')
sys.stdout = f

print(' \n'.join(str_list))

sys.stdout = orig_stdout
f.close()
sys.stdout = orig_stdout

In [ ]:
######## I am done ################3